In [ ]:
# Step 1 Load the Excel data
import pandas as pd
file_path = "games.xlsx"
data = pd.read_excel(file_path)

# Step 2  Initialize ELO Parameters
K = 32 #K factor for ELO rating system
initial_elo = 1500 #Initial ELO rating for all players

# Step 3 variables
point_scale = 20 #You can change this to whatever you want
scaling_factor = 400 #You can change this to whatever you want

# Step 3.5 Define ELO Calculation Functions
import math

def expected_win_probabilty(rating_a, rating_b):
    return 1/(1+math.pow(10, (rating_a-rating_b)/scaling_factor))


def updated_elo(rating_a, rating_b, score_a):
    expected_a = expected_win_probabilty(rating_a, rating_b)
    delta = K*(score_a-expected_a)
    return rating_a+delta

def calculate_metrics(rating_a, rating_b):
    win_prob_a = expected_win_probabilty(rating_a, rating_b)
    win_prob_b = expected_win_probabilty(rating_b, rating_a)
    points_winnable_a = round(point_scale * win_prob_a)
    points_winnable_b = round(point_scale * win_prob_b)
    points_at_risk_a = round(point_scale - points_winnable_a)
    points_at_risk_b = round(point_scale - points_winnable_b)
    points_at_risk_alternate_a = round(point_scale * (1-win_prob_a))
    points_at_risk_alternate_b = round(point_scale * (1-win_prob_b))
    points_winnable_alternate_a = round(point_scale - points_at_risk_alternate_a)
    points_winnable_alternate_b = round(point_scale - points_at_risk_alternate_b)


    return [win_prob_a, win_prob_b, points_winnable_a, points_winnable_b, points_at_risk_a, points_at_risk_b, points_at_risk_alternate_a, points_at_risk_alternate_b, points_winnable_alternate_a, points_winnable_alternate_b]


In [ ]:
# Step 4 Calculate ELO and Metrics


elo_ratings = {player:initial_elo for player in set(data["Agent"].unique())| set(data["Scenario"].unique())}
    

data['Agent Pre-game ELO'] = None
data['Scenario Pre-game ELO'] = None
data['Agent Win Probability'] = None
data['Scenario Win Probability'] = None
data['Agent Post-game ELO'] = None
data['Scenario Post-game ELO'] = None
data['Agent points winnable'] = None
data['Scenario points winnable'] = None
data['Agent points at risk'] = None
data['Scenario points at risk'] = None

for index, row in data.iterrows():
    agent = row['Agent']
    scenario = row['Scenario']
    score = row['Win']

    agent_elo = elo_ratings[agent]
    scenario_elo = elo_ratings[scenario]

    agent_pre_elo = agent_elo
    scenario_pre_elo = scenario_elo

    if score == 'Agent':
        agent_score = 1
        scenario_score = 0
    elif score == 'Scenario':
        agent_score = 0
        scenario_score = 1

    agent_elo = updated_elo(agent_elo, scenario_elo, agent_score)
    scenario_elo = updated_elo(scenario_elo, agent_elo, scenario_score)
    
    elo_ratings[agent] = agent_elo
    elo_ratings[scenario] = scenario_elo
    
    win_prob_agent, win_prob_scenario, points_winnable_agent, points_winnable_scenario, points_at_risk_agent, points_at_risk_scenario = calculate_metrics(agent_pre_elo, scenario_pre_elo)
    
    data.at[index, 'Agent Pre-game ELO'] = agent_pre_elo
    data.at[index, 'Scenario Pre-game ELO'] = scenario_pre_elo
    data.at[index, 'Agent Win Probability'] = win_prob_agent
    data.at[index, 'Scenario Win Probability'] = win_prob_scenario
    data.at[index, 'Agent Post-game ELO'] = agent_elo
    data.at[index, 'Scenario Post-game ELO'] = scenario_elo
    data.at[index, 'Agent points winnable'] = points_winnable_agent
    data.at[index, 'Scenario points winnable'] = points_winnable_scenario
    data.at[index, 'Agent points at risk'] = points_at_risk_agent
    data.at[index, 'Scenario points at risk'] = points_at_risk_scenario

In [ ]:
#Step 5 Calculate Prediction Accuracy

accuracy_counts = {i: {'predicted': 0, 'actual': 0} for i in range(0, 100, 10)}

for _, row in data.iterrows():
    predicted_percent = int(row['Agent Win Probability'] * 100)
    actual_winner = row['Outcome']
    
    accuracy_counts[predicted_percent]['predicted'] += 1
    
    if predicted_percent >= 50 and actual_winner == 'Agent':
        accuracy_counts[predicted_percent]['actual'] += 1
    elif predicted_percent < 50 and actual_winner == 'Scenario':
        accuracy_counts[predicted_percent]['actual'] += 1

print("Predicted vs. Actual Accuracy:")
for percent, counts in accuracy_counts.items():
    if counts['predicted'] > 0:
        accuracy = counts['actual'] / counts['predicted']
        print(f"Predicted {percent}-{percent+9}%: Actual Accuracy = {accuracy:.2%}")

In [ ]:
# Step 6: Create a Histogram for Accuracy Counts

import matplotlib.pyplot as plt

predicted_percentages = [percent for percent in accuracy_counts.keys()]
actual_accuracies = [counts['actual'] / counts['predicted'] if counts['predicted'] > 0 else 0 for counts in accuracy_counts.values()]

plt.bar(predicted_percentages, actual_accuracies, width=8, align='edge', color='blue')
plt.xlabel('Predicted Win Percentage')
plt.ylabel('Actual Accuracy')
plt.title('Prediction Accuracy Histogram')
plt.xticks(range(0, 101, 10))
plt.ylim(0, 1)
plt.grid(True)
plt.show()


In [ ]:
# Step 7: Create a Histogram of ELO Ratings

